In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
WORKING_DIR = '.' #os.path.dirname(__file__)
DATA_DIR = os.path.join(WORKING_DIR, 'data', 'smsspamcollection')
DATA_FILE = os.path.join(DATA_DIR, 'SMSSpamCollection.txt')

CLASS = 'class'
TEXT = 'text'

## 1 - 3

Загрузите датасет по ссылке:
http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip

Считайте датасет в Python (можете сразу грузить все в память, выборка
небольшая), выясните, что используется в качестве разделителей и как
проставляются метки классов.

Подготовьте для дальнейшей работы два списка: список текстов в порядке
их следования в датасете и список соответствующих им меток классов.
В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [3]:
def get_data(filename):
    content = pd.read_csv(filename, sep='\t', encoding='utf-8', names=[CLASS, TEXT])
    labels = content[CLASS] != 'ham'
    return content, content[TEXT].values, labels.astype('int')

In [4]:
data, texts, y = get_data(DATA_FILE)
y.shape

(5572,)

In [5]:
data.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 4

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными
настройками, получите из списка текстов матрицу признаков X.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
default_vectorizer = CountVectorizer()
X = default_vectorizer.fit_transform(texts)
X.shape

(5572, 8713)

## 5

Оцените качество классификации текстов с помощью LogisticRegression() с
параметрами по умолчанию, используя
sklearn.cross_validation.cross_val_score и посчитав среднее
арифметическое качества на отдельных fold'ах. Параметр cv задайте
равным 10. В качестве метрики качества используйте f1-меру.
Получившееся качество – ответ в этом пункте.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [9]:
N_SPLITS = 10
SCORING = 'f1'

In [10]:
def test_classifier(clf, X, y, n_splits, scoring):
    scores = cross_val_score(clf, X, y, cv=n_splits, scoring=scoring, n_jobs=-1)
    return np.mean(scores)

In [11]:
print('Mean f1 score:', test_classifier(LogisticRegression(), X, y, N_SPLITS, SCORING))

Mean f1 score: 0.932640298361


## 6
А теперь обучите классификатор на всей выборке и спрогнозируйте с его
помощью класс для следующих сообщений:
"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use
from your phone now! Subscribe6GB"
"FreeMsg: Txt: claim your reward of 3 hours talk time"
"Have you visited the last lecture on physics?"
"Have you visited the last lecture on physics? Just buy this book and you will have
all materials! Only 99\$"
"Only 99\$"
Выпишите через пробел прогнозы классификатора (0 – не спам, 1 – спам)

In [12]:
TEST_TEXT = [
    'FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB',
    'FreeMsg: Txt: claim your reward of 3 hours talk time',
    'Have you visited the last lecture on physics?',
    'Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$',
    'Only 99$'
]

In [13]:
X_test = default_vectorizer.transform(TEST_TEXT)
X_test.shape

(5, 8713)

In [14]:
clf_lr = LogisticRegression()
clf_lr.fit(X, y)
y_pred = clf_lr.predict(X_test)
print(' '.join(y_pred.astype('str')))

1 1 0 0 0


## 7
Задайте в CountVectorizer параметр ngram_range=(2,2), затем
ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях
измерьте получившееся в кросс-валидации значение f1-меры,
округлите до второго знака после точки, и выпишете результаты через
пробел в том же порядке. В данном эксперименте мы пробовали
добавлять в признаки n-граммы для разных диапазонов n - только
биграммы, только триграммы, и, наконец, все вместе - униграммы,
биграммы и триграммы. Обратите внимание, что статистики по
биграммам и триграммам намного меньше, поэтому классификатор
только на них работает хуже. В то же время это не ухудшает результат
сколько-нибудь существенно, если добавлять их вместе с
униграммами, т.к. за счет регуляризации линейный классификатор не
склонен сильно переобучаться на этих признаках.

In [15]:
VECTORIZERS = [
    CountVectorizer(ngram_range=(2,2)),
    CountVectorizer(ngram_range=(3,3)),
    CountVectorizer(ngram_range=(1,3))
]

In [16]:
def test_vectorizers(vectorizers, texts_raw, y_raw, clf, n_splits, scoring):
    scores = []
    for v in vectorizers:
        X_temp = v.fit_transform(texts_raw)
        scores.append(test_classifier(clf, X_temp, y_raw, n_splits, scoring))
    return np.array(scores).round(decimals = 2)

In [17]:
print(' '.join(test_vectorizers(
            VECTORIZERS, texts, y, LogisticRegression(), N_SPLITS, SCORING).astype('str')))

0.82 0.73 0.93


## 8
Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [18]:
from sklearn.naive_bayes import MultinomialNB

In [19]:
print(' '.join(test_vectorizers(
            VECTORIZERS, texts, y, MultinomialNB(), N_SPLITS, SCORING).astype('str')))

0.65 0.38 0.89


## 9
Попробуйте использовать в логистической регрессии в качестве признаков Tf\*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? Обратите внимание, что результат
перехода к tf\*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
print('Mean f1 score:', test_classifier(
        LogisticRegression(), TfidfVectorizer().fit_transform(texts), y, N_SPLITS, SCORING))

Mean f1 score: 0.852859955417


## 10\*
Попробуйте получить как можно более высокое качество на кросс-валидации. Напишите, что пробовали и какое качество получилось.

#### Grid search для разных комбинаций признаков и классификаторов

In [22]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline, FeatureUnion

from tools import ExtendedModel, ClfTester

In [23]:
FEATURES = {
    'Tf*idf': ExtendedModel(TfidfVectorizer()),
    'Token counts' : ExtendedModel(CountVectorizer()),
    'Tf*idf & token counts': ExtendedModel(FeatureUnion([
        ('Tf*idf', TfidfVectorizer()),
        ('Token counts', CountVectorizer())
    ]))
}

In [24]:
CLASSIFIERS = {
    'Logistic Regression' : ExtendedModel( 
        LogisticRegression(), 
        {
            'penalty': ('l1', 'l2'), 
            'C': [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]
        }
    ),
    'Naive Bayes': ExtendedModel(
        MultinomialNB(),
        {
            'alpha': [10, 1, 0.1, 0.01, 0.001, 0.0001]
        }
    ),
    'Linear SVM' : ExtendedModel(
        LinearSVC(),
        {
            'C': [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]
        }
    ),
    'Linear SVM & Features selection' : ExtendedModel(
        Pipeline([
            ('K_best', SelectKBest(chi2, k=500)), 
            ('Linear_SVM', LinearSVC())
        ]),
        {
            'K_best__k': [100, 250, 500, 1000, 2000, 5000],
            'Linear_SVM__C': [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 10000, 15000, 20000, 100000]
        }
    )  
}

In [25]:
clf_tester = ClfTester(FEATURES, CLASSIFIERS, SCORING, N_SPLITS)
df_test_scores, df_best_estimators = clf_tester.test(texts, y, show_time_log=False)

--- Best model ---
Tf*idf + Linear SVM with score 0.9512 and params:
	Classifier__C: 1



In [26]:
df_test_scores.style.highlight_max(axis=None, subset=pd.IndexSlice[:, :])

#### С фильтрацией стоп слов из sklearn

In [27]:
FEATURES = {
    'Tf*idf': ExtendedModel(TfidfVectorizer(stop_words='english')),
    'Token counts' : ExtendedModel(CountVectorizer(stop_words='english')),
    'Tf*idf & token counts': ExtendedModel(FeatureUnion([
        ('Tf*idf', TfidfVectorizer(stop_words='english')),
        ('Token counts', CountVectorizer(stop_words='english'))
    ]))
}

In [28]:
clf_tester = ClfTester(FEATURES, CLASSIFIERS, SCORING, N_SPLITS)
df_test_scores, df_best_estimators = clf_tester.test(texts, y, show_time_log=False)

--- Best model ---
Tf*idf + Linear SVM with score 0.9468 and params:
	Classifier__C: 10



In [29]:
df_test_scores.style.highlight_max(axis=None, subset=pd.IndexSlice[:, :])

#### С фильтрацией стоп слов из nltk

In [30]:
from nltk.corpus import stopwords

In [31]:
NLTK_STOP_WORDS = list(stopwords.words('english'))

FEATURES = {
    'Tf*idf': ExtendedModel(TfidfVectorizer(stop_words=NLTK_STOP_WORDS)),
    'Token counts' : ExtendedModel(CountVectorizer(stop_words=NLTK_STOP_WORDS)),
    'Tf*idf & token counts': ExtendedModel(FeatureUnion([
        ('Tf*idf', TfidfVectorizer(stop_words=NLTK_STOP_WORDS)),
        ('Token counts', CountVectorizer(stop_words=NLTK_STOP_WORDS))
    ]))
}

In [32]:
clf_tester = ClfTester(FEATURES, CLASSIFIERS, SCORING, N_SPLITS)
df_test_scores, df_best_estimators = clf_tester.test(texts, y, show_time_log=False)

--- Best model ---
Tf*idf + Logistic Regression with score 0.9462 and params:
	Classifier__penalty: l2
	Classifier__C: 100000



In [33]:
df_test_scores.style.highlight_max(axis=None, subset=pd.IndexSlice[:, :])

#### С токенизацией слов из nltk

In [34]:
from nltk.tokenize import word_tokenize

In [35]:
def nltk_tokenize(texts_raw):
    new_texts = []
    for text in texts_raw:
        tokens = word_tokenize(text)
        new_texts.append(' '.join(tokens))
    return new_texts

In [36]:
texts_nltk_tokens = nltk_tokenize(texts)

In [37]:
FEATURES = {
    'Tf*idf': ExtendedModel(TfidfVectorizer(analyzer=str.split)),
    'Token counts' : ExtendedModel(CountVectorizer(analyzer=str.split)),
    'Tf*idf & token counts': ExtendedModel(FeatureUnion([
        ('Tf*idf', TfidfVectorizer(analyzer=str.split)),
        ('Token counts', CountVectorizer(analyzer=str.split))
    ]))
}

In [38]:
clf_tester = ClfTester(FEATURES, CLASSIFIERS, SCORING, N_SPLITS)
df_test_scores, df_best_estimators = clf_tester.test(texts_nltk_tokens, y, show_time_log=False)

--- Best model ---
Token counts + Naive Bayes with score 0.9546 and params:
	Classifier__alpha: 0.1



In [39]:
df_test_scores.style.highlight_max(axis=None, subset=pd.IndexSlice[:, :])

#### С токенизацией и лемматизацией из nltk

In [40]:
from nltk.stem import WordNetLemmatizer

In [57]:
def nltk_lemmatize(texts_raw, with_stop_words=False):
    wnl = WordNetLemmatizer()
    new_texts = []
    for text in texts_raw:
        tokens = [wnl.lemmatize(t) for t in word_tokenize(text)]
        if with_stop_words:
            tokens = [t for t in tokens if t not in stopwords.words('english')]
        new_texts.append(' '.join(tokens))
    return new_texts

In [42]:
texts_nltk_lemms = nltk_lemmatize(texts)

In [43]:
clf_tester = ClfTester(FEATURES, CLASSIFIERS, SCORING, N_SPLITS)
df_test_scores, df_best_estimators = clf_tester.test(texts_nltk_lemms, y, show_time_log=False)

--- Best model ---
Token counts + Naive Bayes with score 0.952 and params:
	Classifier__alpha: 0.1



In [44]:
df_test_scores.style.highlight_max(axis=None, subset=pd.IndexSlice[:, :])

#### С токенизацией и стеммингом из nltk

In [45]:
from nltk.stem import PorterStemmer

In [46]:
def nltk_stem(texts_raw, with_stop_words=False):
    ps = PorterStemmer()
    new_texts = []
    for text in texts_raw:
        tokens = [ps.stem(t) for t in word_tokenize(text)]
        if with_stop_words:
            tokens = [t for t in tokens if t not in stopwords.words('english')]
        new_texts.append(' '.join(tokens))
    return new_texts

In [51]:
texts_nltk_stems = nltk_stem(texts)

In [52]:
clf_tester = ClfTester(FEATURES, CLASSIFIERS, SCORING, N_SPLITS)
df_test_scores, df_best_estimators = clf_tester.test(texts_nltk_stems, y, show_time_log=False)

--- Best model ---
Token counts + Naive Bayes with score 0.9503 and params:
	Classifier__alpha: 0.1



In [53]:
df_test_scores.style.highlight_max(axis=None, subset=pd.IndexSlice[:, :])

#### С токенизацией, стеммингом и стоп словами из nltk

In [54]:
texts_nltk_stems_stop_words = nltk_stem(texts, with_stop_words=True)

In [55]:
clf_tester = ClfTester(FEATURES, CLASSIFIERS, SCORING, N_SPLITS)
df_test_scores, df_best_estimators = clf_tester.test(texts_nltk_stems_stop_words, y, show_time_log=False)

--- Best model ---
Token counts + Naive Bayes with score 0.9491 and params:
	Classifier__alpha: 0.1



In [56]:
df_test_scores.style.highlight_max(axis=None, subset=pd.IndexSlice[:, :])

#### С токенизацией, лемматизацией и стоп словами из nltk

In [58]:
texts_nltk_lemms = nltk_lemmatize(texts, with_stop_words=True)

In [59]:
clf_tester = ClfTester(FEATURES, CLASSIFIERS, SCORING, N_SPLITS)
df_test_scores, df_best_estimators = clf_tester.test(texts_nltk_lemms, y, show_time_log=False)

--- Best model ---
Token counts + Naive Bayes with score 0.949 and params:
	Classifier__alpha: 0.1



In [60]:
df_test_scores.style.highlight_max(axis=None, subset=pd.IndexSlice[:, :])

## 11
Какие наблюдения и выводы можно сделать из этого задания?

На исследуемых данных лучше всего себя показал наивный байесовский классификатор на униграмах, полученнных с использованием токенизатора из библиотеки nltk.

Общий вывод: на любых данных нужно пытаться проверить различные алгоритмы классификации в сочетании с различными методами предобработки данных и различными признаковыми пространствами, и не стоит рассчитывать, что на разных данных лучший метод будет один и тот же. 